# Simulate the CT scan

In [ ]:
import os
import matplotlib.pyplot as plt
from JSON2gVXRDataReader import *

import SimpleITK as sitk

from cil.utilities.display import show_geometry
from cil.processors import TransmissionAbsorptionConverter
from cil.recon import FDK
from cil.utilities.jupyter import islicer, link_islicer

In [ ]:
current_folder = str(globals()['_dh'][0])
data_path = "25OctoberData/"
data_path = "24NovemberData/"

simulation_JSON_fname = current_folder + "/" + data_path + "/simulation.json"
visible_light_JSON_fname = current_folder + "/" + data_path + "/visible_light.json"

In [ ]:
def reconstruct(JSON_fname):
    reader = JSON2gVXRDataReader(file_name=JSON_fname)
    data = reader.read(verbose=0)
    data.reorder(order='tigre')
    data_corr = TransmissionAbsorptionConverter(white_level=data.max(), min_intensity=1e-5)(data)
    ig = data.geometry.get_ImageGeometry()
    recons = FDK(data_corr, ig).run()
    
    return data, ig, recons

def save(fname, recons):
# detector_size = np.array(gvxr.getDetectorSize("mm"))
# number_of_pixels = np.array(gvxr.getDetectorNumberOfPixels())
# spacing = detector_size / number_of_pixels

# print("CT volume saved in", "recons.mha")

    reconstruction_as_array = recons.as_array()
    sitk_image = sitk.GetImageFromArray(reconstruction_as_array)
    # sitk_image.SetSpacing([spacing[0], spacing[0], spacing[1]])
    sitk.WriteImage(sitk_image, fname, useCompression=True)


In [ ]:
simulation_data, simulation_ig, simulation_recons = reconstruct(simulation_JSON_fname)

In [ ]:
visible_light_data, visible_light_ig, visible_light_recons = reconstruct(visible_light_JSON_fname)

In [ ]:
print(simulation_data.geometry)

In [ ]:
show_geometry(simulation_data.geometry).save(current_folder + "/" + data_path + "/geometry.png")

In [ ]:
sl1 = islicer(simulation_recons, direction='vertical')
sl2 = islicer(visible_light_recons, direction='vertical')
link_islicer(sl1, sl2)

In [ ]:
sl3 = islicer(simulation_recons, direction='horizontal_x')
sl4 = islicer(visible_light_recons, direction='horizontal_x')
link_islicer(sl3, sl4)

In [ ]:
# save(current_folder + "/" + data_path + "/simulation.mha", simulation_recons)

In [ ]:
# save(current_folder + "/" + data_path + "/visible_light.mha", visible_light_recons)